In [3]:
import numpy as np

from tools.kernels import kernel_matrix
from tools.optimization import find_f
from tools.prediction import pred

In [ ]:
X = np.array([[1, 2,5], [1,2, 4], [-5, -4, -3], [-5, -7, 0]])

In [ ]:
K = kernel_matrix(X)

In [ ]:
Y = [1, 1, 0, 0]

In [ ]:
alpha = find_f(K, Y, lamb=0.1)

In [ ]:
alpha

In [ ]:
X_test = np.array([[1,2,3], [-5, -6, -2]])

In [ ]:
pred(X, X_test, alpha)

In [4]:
Y_pred = np.array([[0.1, 0.2, -0.4], [0.4, 0.01, 0.3]])

In [5]:
Y_pred

array([[ 0.1 ,  0.2 , -0.4 ],
       [ 0.4 ,  0.01,  0.3 ]])

In [7]:
np.argmax(Y_pred, axis=1)

array([1, 0])